In [1]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 927.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Prep dataset

In [3]:
# prepare data
from datasets import load_dataset
eval_dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:1%]")

README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

In [4]:
eval_dataset

Dataset({
    features: ['Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 406
})

In [5]:
import pandas as pd
eval_dataset = pd.DataFrame(eval_dataset)
eval_dataset.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


In [ ]:
# prompt_template = "You are an experienced medical professional. Answer the following medial question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: {}"

# eval_dataset['prompt'] = eval_dataset.apply(lambda x: prompt_template.format(x['Open-ended Verifiable Question']), axis=1)
# eval_dataset.prompt[0]

In [ ]:
# from datasets import Dataset
# ds = Dataset.from_pandas(eval_dataset[['prompt']])
# ds

## Set up model definition

In [ ]:
# with open('/content/drive/My Drive/w266/project/266 Final Project/llama3pt2-1b-peft/adapter_config.json', 'r') as f:
#   l = f.readlines()

In [6]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the LoRA model
model_id = "erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quantization_config,
#     token=userdata.get('HF_TOKEN'))

In [7]:
# Load tokenizer
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True,
                                          token=userdata.get('HF_TOKEN'))
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

1

In [8]:
tokenizer.get_chat_template()

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

## Eval of LoRA SFT (R = 32) model

In [9]:
# set up inference pipeline
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_id,
                model_kwargs={
                    "quantization_config":quantization_config,
                },
                token=userdata.get('HF_TOKEN'))


adapter_config.json:   0%|          | 0.00/917 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
fn_out = '/content/drive/MyDrive/w266_project/lora-sft-r32-output-reformatted.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

In [ ]:
for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):
  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content": "Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample}
  ]
  # prompt = tokenizer.apply_chat_template(prompt, tokenize=False)

  # prompt = "You are a medical expert. Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample
  for r in range(5):
    outputs = pipe(
        prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
    response = outputs[0]["generated_text"]
    response = response.replace('\n','\t')
    print(response)
    with open(fn_out, 'a') as f:
        f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 100:
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of the gastrointestinal blood loss in this 88-year-old woman, considering her use of naproxen, is an ulcer. Naproxen, a nonsteroidal anti-inflammatory drug (NSAID), can cause erosive lesions in the stomach lining, leading to ulcers. These ulcers can bleed, potentially resulting in the coffee-ground vomit that she is experiencing.		Rating the correctness of this answer: 1. Correct.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of the gastrointestinal blood loss in this 88-year-old woman, who is experiencing mild epigastric discomfort and vomiting material resembling coffee grounds, is peptic ulcers. These ulcers can lead to gastrointestinal bleeding and are potentially exacerbated by her use of naproxen, a nonsteroidal anti-inflammatory drug (NSAID) commonly used for the management of osteoarthritis. Thus, the most likely cause of the gastrointestinal blood loss is peptic ulcers caused by naproxen use.		Rating the correctness of this answer: 1. Correct.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Based on the symptoms described, the most likely diagnosis for this condition is "Scabies." Scabies is a common and highly contagious skin infection that presents with scaly lesions typically located in areas where fingers can scratch them. The lesions of scabies often exhibit a specific pattern, commonly referred to as "buses" or "tracks." The healing of these lesions often leads to post-inflammatory hyperpigmentation, which can align with the patient's experience of darkening of the skin in that area.		Rating: 1) Correct


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely diagnosis for this condition is fixed condyloma acuminatum. This type of lesion tends to occur at the same site on the penis, presents as a scaly, flat growth, and features hyperpigmentation upon healing.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely diagnosis for this condition is fixed condyloma. These are common warts on the penis that occur at specific sites, often at the glans, and have characteristic appearances and healing patterns.		Correctness rating: 1) Correct.
Based on the symptoms described, the most likely diagnosis is condyloma acuminatum. This is a common viral condition that often appears on the genitalia and presents with the symptoms described: scaly lesions that recur at the same location and resolve with some hyperpigmentation. 		The correctness of this answer is: 1. Correct.


In [10]:
# prepare data for bootstrapping
fn_out = '/content/drive/MyDrive/w266_project/lora-sft-r32-output-reformatted-bootstrapping.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):
  if idx <= 101:
    continue
  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content":sample}
  ]
  # prompt = tokenizer.apply_chat_template(prompt, tokenize=False)

  # prompt = "You are a medical expert. Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample
  for r in range(5):
    outputs = pipe(
        prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
    response = outputs[0]["generated_text"]
    response = response.replace('\n','\t')
    print(response)
    with open(fn_out, 'a') as f:
        f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 201:
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In this case, the medication most likely to be beneficial for the child's behavioral symptoms is a type of antipsychotic, specifically olanzapine. Olanzapine is known for its effectiveness in managing behaviors associated with irritability, aggression, and mood fluctuations, particularly in children with autism. It's also important to consider a thorough evaluation by a healthcare professional to confirm the appropriate diagnosis, as the child's symptoms align with some features of Autism Spectrum Disorder (ASD), which is often associated with these behaviors. Once the diagnosis is confirmed, olanzapine can be considered as a potential medication option to help manage the child's symptoms effectively.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The symptoms described in the 11-year-old boy—swearing outbursts, facial contortions, repetitive blinking, and grunting—suggest an underlying condition characterized by significant behavioral and emotional difficulties, likely autism spectrum disorder (ASD). In managing these symptoms, especially considering his bullying issues, medication can be beneficial. The most effective and commonly used medication for ASD-related symptoms in children is Risperdal (Risperidone). This medication is known for its ability to help manage behavioral issues such as aggression and repetitive behaviors. However, it's crucial to remember that medication alone is not usually the sole solution, and a comprehensive treatment plan, often including behavioral therapies, is typically the most effective approach. It's advisable to consult with a healthcare expert, such as a psychiatrist, who can assess and manage this child's needs thoroughly.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A 32-year-old primigravida woman who is blood group O positive and is married to a blood group AB positive individual should understand that the ABO blood group incompatibility might result in a more severe hemolytic disease of the newborn (HDN) in their baby. This condition occurs when the mother's antibodies react with the baby's red blood cells, leading to destruction of the cells. Since her husband is AB positive, the mother has the potential for both anti-A and anti-B antibodies, which can combine to form a stronger antibody, making it more challenging for the newborn to produce a sufficient number of healthy red blood cells. This means the newborn might need additional care, such as the exchange transfusion, to help reduce the amount of these harmful antibodies in the baby's blood. While these risks are concerning, she should also be aware that the likelihood of HDN varies depending on whether her husband is AB or B positive and the strength of her antibodies. It is advisable for

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


As a pregnant woman who is blood group O positive and whose husband is blood group AB positive, you should understand that there is no significant risk associated with ABO blood group incompatibility impacting the newborn's health in this scenario. Blood group AB individuals have both A and B antigens on their red blood cells, which means they can accept both A and B antigens without an adverse reaction. This characteristic of AB blood makes them less likely to have a strong immune response if they inherit an A or B antigen from their child, which is what typically occurs when a child inherits an A or B antigen from either parent. Therefore, as you have an AB genotype due to your husband's AB positive blood, your child will not receive any AB antigens from either parent and thus there are no issues with ABO incompatibility to consider.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


When a pregnant woman with a blood type of O positive and her partner with AB positive are expecting a child, there is a potential risk of ABO blood group incompatibility. This incompatibility can occur due to the mother's immune system developing antibodies against the father's AB antigens present on the newborn's cells. This can lead to Hemolytic Disease of the Newborn (HDN), also known as Rh disease, if the mother has Rh-negative blood, especially if her husband is blood type AB.		In this scenario, the mother will likely have developed antibodies against the A and B antigens present in her partner's blood. This would result in a transfer of these antibodies across the placenta to the fetus. In newborns, this can lead to the destruction of their own red blood cells, which can cause a range of symptoms, from mild to severe, depending on the level of antibody transmission. The risk of HDN increases when these maternal antibodies are present, especially if the fetus is also Rh-negative.